# 处理product信息

In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
import random

In [2]:
product_order = pd.read_csv('./product_order.csv')

In [3]:
product_order.drop(['game_id', 'channel_id', 'source_id','server_id', 'order_no','d_discount','d_money', 's_discount',
                    's_money','status','role_name'], axis=1, inplace=True)
print(product_order.shape)
product_order.head()

(1641426, 18)


,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt
0,12023872,106335096,0,12.0,0.0,12.0,0.0,0.0,NaN,0,1,1,55,32151CD0-65C1-4B05-91D7-367E8431C214,2020-03-13 16:45:02,2020-03-13 16:45:02,1,2020-03-13
1,12043074,106434579,0,128.0,0.0,128.0,0.0,0.0,NaN,0,1,1,53,BFD0278A-815B-4900-B3EB-AF106FAF66F0,2020-03-13 13:45:01,2020-03-13 13:45:01,1,2020-03-13
2,11467501,103598817,27,68.0,0.0,68.0,0.0,0.0,NaN,0,35,35,0,265CD2C8-A2A5-4ECE-9CFB-CC2AA195F0F1,2020-03-13 00:00:33,2020-03-13 00:00:33,35,2020-03-13
3,12063758,106557590,99,68.0,0.0,68.0,0.0,0.0,NaN,0,2,2,1,18C3BAF2-16E0-48AC-9999-8250815D8362,2020-03-13 00:01:21,2020-03-13 00:01:21,2,2020-03-13
4,11467501,103598817,27,68.0,0.0,68.0,0.0,0.0,NaN,0,36,36,0,265CD2C8-A2A5-4ECE-9CFB-CC2AA195F0F1,2020-03-13 00:01:10,2020-03-13 00:01:10,36,2020-03-13


In [4]:
# 到目前（2020-6-30）为此
print('总流水收入:',product_order['rmb_money'].sum())
print('平均每单付费金额:', product_order['rmb_money'].mean())
print('付费玩家数:', product_order['user_id'].nunique())
print('付费角色数:', product_order['role_id'].nunique())

总流水收入: 221038877.06000006
平均每单付费金额: 134.66271221484322
付费玩家数: 144094
付费角色数: 160238


In [5]:
groups = product_order.groupby('role_id')
temp = groups['rmb_money'].agg([('sum_money', 'sum')]).sort_values('sum_money', ascending=False)

In [6]:
temp.head(10)

,sum_money
role_id,
10072297,686661.00
10953716,673088.70
12070838,531640.42
11799188,384122.36
11281247,378620.90
10106322,362641.00
10976235,362639.56
135336493,339988.00
133140009,325070.00


In [7]:
# 将所有付费过的user_id标记为1
product_order['is_pay'] = 1

In [8]:
# 根据user_id,role_id把缺失的device_no补全，众数补全
def mode_1(x):
    mode_list = x.mode()
    if len(mode_list) ==1:
        mode = mode_list[0]
    if len(mode_list) ==0:
        mode = np.nan
    if len(mode_list)>1:
        mode = random.choice(mode_list)
    return mode
device_no_filler = pd.DataFrame(product_order.groupby(['role_id'])['device_no'].apply(mode_1), columns=['device_no'])
product_order.update(device_no_filler, overwrite=False)

In [9]:
product_order['pay_time'] = pd.to_datetime(product_order['pay_time'])

In [10]:
role_create_time = pd.read_csv('./FE_data/users_info_all.csv', usecols=['role_id', 'create_role_time'])
role_create_time['create_role_time'] = pd.to_datetime(role_create_time['create_role_time'])
role_create_time.head()

,role_id,create_role_time
0,11653263,2020-02-26 00:00:08
1,11653264,2020-02-26 00:00:18
2,11653265,2020-02-26 00:00:18
3,11653267,2020-02-26 00:00:19
4,11653269,2020-02-26 00:00:28


In [11]:
product_order.shape

(1641426, 19)

In [12]:
# 末日所有订单信息保存
product_order.to_csv('./FE_data/product_order.csv', index=None)

In [13]:
# 选取角色注册7天的订单
temp = pd.merge(product_order, role_create_time, on='role_id', how='left')

In [14]:
temp.head()

,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,create_role_time
0,12023872,106335096,0,12.0,0.0,12.0,0.0,0.0,NaN,0,1,1,55,32151CD0-65C1-4B05-91D7-367E8431C214,2020-03-13 16:45:02,2020-03-13 16:45:02,1,2020-03-13,1,2020-03-10 01:27:09
1,12043074,106434579,0,128.0,0.0,128.0,0.0,0.0,NaN,0,1,1,53,BFD0278A-815B-4900-B3EB-AF106FAF66F0,2020-03-13 13:45:01,2020-03-13 13:45:01,1,2020-03-13,1,2020-03-10 22:00:37
2,11467501,103598817,27,68.0,0.0,68.0,0.0,0.0,NaN,0,35,35,0,265CD2C8-A2A5-4ECE-9CFB-CC2AA195F0F1,2020-03-13 00:00:33,2020-03-13 00:00:33,35,2020-03-13,1,2020-02-18 01:44:52
3,12063758,106557590,99,68.0,0.0,68.0,0.0,0.0,NaN,0,2,2,1,18C3BAF2-16E0-48AC-9999-8250815D8362,2020-03-13 00:01:21,2020-03-13 00:01:21,2,2020-03-13,1,2020-03-11 21:52:44
4,11467501,103598817,27,68.0,0.0,68.0,0.0,0.0,NaN,0,36,36,0,265CD2C8-A2A5-4ECE-9CFB-CC2AA195F0F1,2020-03-13 00:01:10,2020-03-13 00:01:10,36,2020-03-13,1,2020-02-18 01:44:52


In [15]:
# temp['interval'] = ((temp['pay_time']-temp['create_role_time']).dt.seconds + 
#                     (temp['pay_time']-temp['create_role_time']).dt.days*24*3600)/3600
temp['interval'] = (temp['pay_time'] - temp['create_role_time']).dt.total_seconds()/3600

In [16]:
temp.sort_values('interval', ascending=False).head()

,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,...,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,create_role_time,interval
1636040,128580549,87789863,25,98.0,0.0,98.0,0.0,0.0,NaN,0,...,36,262,82FBA763ACC98219AC02BCD01107A5AF,2020-06-29 00:47:40,2020-06-29 00:47:40,12,2020-06-29,1,2019-07-24 14:05:19,8170.705833
1637313,128900655,88118939,2,68.0,0.0,68.0,0.0,0.0,NaN,0,...,107,270,DCB51C4D85F688341D3F48B2A024C87E,2020-06-29 13:35:58,2020-06-29 13:35:58,107,2020-06-29,1,2019-07-29 05:05:12,8072.512778
1636416,128890643,88109664,100,648.0,0.0,648.0,0.0,0.0,0,0,...,81,274,75A9BCD650EAD0CF857416E0204A2709,2020-06-29 05:04:23,2020-06-29 05:04:23,80,2020-06-29,1,2019-07-28 22:44:28,8070.331944
1636402,128890643,88109664,100,198.0,0.0,198.0,0.0,0.0,0,0,...,80,274,75A9BCD650EAD0CF857416E0204A2709,2020-06-29 04:44:34,2020-06-29 04:44:34,79,2020-06-29,1,2019-07-28 22:44:28,8070.001667
1514042,128783885,88002783,25,30.0,0.0,30.0,0.0,0.0,NaN,0,...,55,274,60B1ED55070B0718C5B13EB127C19388,2020-06-27 10:10:56,2020-06-27 10:10:56,55,2020-06-27,1,2019-07-27 16:08:46,8058.036111


In [17]:
temp['interval'] = pd.to_numeric(temp['interval'], errors='coerce')
day_7_product = temp[temp['interval'] <= 168]

In [18]:
day_7_product.shape

(661633, 21)

In [19]:
day_7_product['rmb_money'].sum()

52900824.82

In [20]:
day_7_product.to_csv('./FE_data/day_7_product.csv', index=None)

In [21]:
# 导出角色注册前3天的订单
day_3_product = temp[temp['interval'] <= 72]
print(day_3_product.shape)
print(day_3_product['rmb_money'].sum())
day_3_product.to_csv('./FE_data/day_3_product.csv', index=None)

(490918, 21)
32509295.26


In [22]:
# 导出角色注册前30天的订单
# temp = pd.merge(product_order, role_create_time, on='role_id', how='left')
# temp['interval'] = (temp['pay_time'] - temp['create_role_time']).dt.total_seconds()/3600
# temp['interval'] = pd.to_numeric(temp['interval'], errors='coerce')
day_30_product = temp[temp['interval'] <= 720]
print(day_30_product.shape)
print(day_30_product['rmb_money'].sum())
day_30_product.to_csv('./FE_data/day_30_product.csv', index=None)

(1081669, 21)
121540858.25999998


In [23]:
groups = day_7_product.groupby(['role_id'])
# 前7天充值总次数  p1
temp = groups.size().reset_index().rename(columns={0:'p1'})
matrix_1 = pd.merge(day_7_product, temp, on='role_id', how='left', validate='many_to_one')

# 前7天充值总金额  p2
temp = groups['money'].agg([('p2', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前7天充值give_money次数 p3
temp = pd.DataFrame(groups['give_money'].apply(np.count_nonzero)).reset_index().rename(columns={'give_money':'p3'})
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')
# # 前7天充值give_money总金额 p4
temp = groups['give_money'].agg([('p4', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前7天充值rmb_money次数 p5
temp = pd.DataFrame(groups['rmb_money'].apply(np.count_nonzero)).reset_index().rename(columns={'rmb_money':'p5'})
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')

# 前7天充值rmb_money总金额 p6
temp = groups['rmb_money'].agg([('p6', 'sum')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='role_id', how='left', validate='many_to_one')


In [24]:
pd.set_option('display.max_columns', None)
matrix_1.head()

,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,create_role_time,interval,p1,p2,p3,p4,p5,p6
0,12023872,106335096,0,12.0,0.0,12.0,0.0,0.0,NaN,0,1,1,55,32151CD0-65C1-4B05-91D7-367E8431C214,2020-03-13 16:45:02,2020-03-13 16:45:02,1,2020-03-13,1,2020-03-10 01:27:09,87.298056,1,12.0,0,0.0,1,12.0
1,12043074,106434579,0,128.0,0.0,128.0,0.0,0.0,NaN,0,1,1,53,BFD0278A-815B-4900-B3EB-AF106FAF66F0,2020-03-13 13:45:01,2020-03-13 13:45:01,1,2020-03-13,1,2020-03-10 22:00:37,63.740000,4,176.0,0,0.0,4,176.0
2,12063758,106557590,99,68.0,0.0,68.0,0.0,0.0,NaN,0,2,2,1,18C3BAF2-16E0-48AC-9999-8250815D8362,2020-03-13 00:01:21,2020-03-13 00:01:21,2,2020-03-13,1,2020-03-11 21:52:44,26.143611,10,540.0,0,0.0,10,540.0
3,12039463,106412872,2,30.0,0.0,30.0,0.0,0.0,NaN,0,3,3,0,76FF7593-8A26-454F-9A8E-7591FF81E5DD,2020-03-13 00:01:46,2020-03-13 00:01:46,3,2020-03-13,1,2020-03-10 19:19:25,52.705833,5,188.0,0,0.0,5,188.0
4,12084747,105791073,2,68.0,0.0,68.0,0.0,0.0,NaN,0,10,5,0,456AB5A6-DBC3-4B86-B624-487ED2E94E35,2020-03-13 00:02:02,2020-03-13 00:02:02,10,2020-03-13,1,2020-03-12 19:58:21,4.061389,14,754.0,0,0.0,14,754.0


In [25]:
# 选择一些字段进行保存
product_info_7d = matrix_1[['user_id','role_id','is_pay','p1','p2','p3','p4','p5','p6']]
product_info_7d = product_info_7d.drop_duplicates()

In [26]:
product_info_7d.to_csv('./FE_data/product_info_7d.csv', index=None)

# 处理login信息

In [27]:
login_info_7d = pd.read_csv('./FE_data/login_info_7d.csv')

In [28]:
# 压缩数据
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [29]:
login_info_7d = reduce_mem_usage(login_info_7d)

Mem. usage decreased to 2802.88 Mb (17.3% reduction)


In [30]:
login_info_7d.drop('b.role_id', axis=1, inplace=True)
new_columns = [columns[2:] for columns in login_info_7d.columns]
login_info_7d.columns = new_columns
login_info_7d.head()

,user_id,role_id,login_time,role_level,device_no,ip,net_w,p_model,sys_ver,sdk_no,dt,create_role_time
0,96918420,10027198,2019-11-03 16:53:49,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
1,96918420,10027198,2019-11-03 16:50:52,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
2,96918420,10027198,2019-11-03 16:53:11,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
3,96918420,10027198,2019-11-03 16:54:42,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
4,96918420,10027198,2019-11-03 16:55:36,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00


In [31]:
# groups = login_info_7d.groupby('role_id')
# # 角色7天内登陆频次
# temp = groups.size().reset_index().rename(columns = {0:'login_num'}).sort_values('login_num', ascending=False)
# temp = temp[temp['login_num'] == 1]
# print('总角色数:',login_info_7d['role_id'].nunique())
# print('7天内只登陆过1次的角色数:', temp['role_id'].nunique())

In [32]:
# # 注册7天内只登陆过1次的角色，全部付费情况
# temp_product = product_order[product_order['role_id'].isin(temp['role_id'])]
# print(temp_product['role_id'].nunique())
# print(temp_product['rmb_money'].sum())

In [33]:
# # 注册7天内只登陆过1次的角色，前7天的付费情况
# temp_product_7d = day_7_product[day_7_product['role_id'].isin(temp['role_id'])]
# print(temp_product_7d['role_id'].nunique())
# print(temp_product_7d['rmb_money'].sum())

In [34]:
# 删除注册7天只登陆过1次的角色
# login_info_7d.drop(login_info_7d[login_info_7d['role_id'].isin(temp['role_id'])].index, inplace=True)

In [35]:
# login_info_7d['role_id'].nunique()

In [36]:
login_info_7d.head()

,user_id,role_id,login_time,role_level,device_no,ip,net_w,p_model,sys_ver,sdk_no,dt,create_role_time
0,96918420,10027198,2019-11-03 16:53:49,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
1,96918420,10027198,2019-11-03 16:50:52,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
2,96918420,10027198,2019-11-03 16:53:11,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
3,96918420,10027198,2019-11-03 16:54:42,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00
4,96918420,10027198,2019-11-03 16:55:36,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00


In [37]:
# 角色7天内登陆的设备数，删除登陆设备大于等于8的role_id
groups = login_info_7d.groupby('role_id')
temp = groups['device_no'].agg([('device_no_num', 'nunique')]).reset_index().sort_values('device_no_num', ascending=False)
temp = temp[temp['device_no_num']>=8]
# 登陆设备大于等于8的role_id付费总金额
print('登陆设备大于等于8的role_id付费总金额:' ,product_order[product_order['role_id'].isin(temp['role_id'])]['rmb_money'].sum())
login_info_7d.drop(login_info_7d[login_info_7d['role_id'].isin(temp['role_id'])].index, inplace=True)

登陆设备大于等于8的role_id付费总金额: 0.0


In [38]:
login_info_7d.isna().sum()

user_id                    0
role_id                    0
login_time                 0
role_level                 0
device_no                  0
ip                         0
net_w               19421428
p_model                    1
sys_ver                    0
sdk_no                     0
dt                         0
create_role_time           0
dtype: int64

In [39]:
# 根据role_id把缺失的p_model补全，众数补全
# net_w_filler = pd.DataFrame(login_info_7d.groupby(['role_id'])['net_w'].apply(mode_1), columns=['net_w'])
# login_info_7d.update(net_w_filler, overwrite=False)

In [40]:
# 按uesr_id,role_id进行聚合，查看user_id,role_id交互行为
groups = login_info_7d.groupby(['user_id','role_id'])
# user_id, role_id本周登录次数 r1
temp = groups.size().reset_index().rename(columns={0:'r1'})
matrix = pd.merge(login_info_7d, temp, on=['user_id','role_id'], how='left')
# user_id, role_id本周登录的设备数r2
temp = groups['device_no'].agg([('r2', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# user,role_id本周使用的ip数r3
temp = groups['ip'].agg([('r3', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# user,role_id本周登录的平均时间间隔r4 按小时
login_info_7d['login_time'] = pd.to_datetime(login_info_7d['login_time'])
temp = groups['login_time'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['r4'] = round((temp['L_time']-temp['F_time']).dt.total_seconds()/3600, 2)
matrix = pd.merge(matrix, temp[['user_id','role_id', 'r4']], on=['user_id','role_id'], how='left')
matrix['r4'] = round(matrix['r4']/(matrix['r1']+1e-4),2) 

In [41]:
# 角色7天内最大等级 max_role_level
groups = login_info_7d.groupby(['user_id','role_id'])
temp = groups['role_level'].agg([('max_role_level', 'max')]).reset_index()
matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')

In [42]:
matrix.head()

,user_id,role_id,login_time,role_level,device_no,ip,net_w,p_model,sys_ver,sdk_no,dt,create_role_time,r1,r2,r3,r4,max_role_level
0,96918420,10027198,2019-11-03 16:53:49,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00,8,1,1,0.01,1
1,96918420,10027198,2019-11-03 16:50:52,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00,8,1,1,0.01,1
2,96918420,10027198,2019-11-03 16:53:11,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00,8,1,1,0.01,1
3,96918420,10027198,2019-11-03 16:54:42,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00,8,1,1,0.01,1
4,96918420,10027198,2019-11-03 16:55:36,1,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,223.104.95.77,NaN,iPhone 8 Plus,13.1.3,1.7.0,2019-11-03,2019-11-03 16:50:00,8,1,1,0.01,1


In [43]:
# 选取部分字段
login_7d_info = matrix[['user_id', 'role_id','r1', 'r2', 'r3', 'r4', 'max_role_level']]
login_7d_info = login_7d_info.drop_duplicates()
print(login_7d_info.shape)
login_7d_info.to_csv('./FE_data/login_7d_info.csv', index=False)

(5951318, 7)


In [44]:
product_login = pd.merge(login_7d_info, product_info_7d, on=['user_id', 'role_id'], how='left', validate='one_to_one')
product_login.fillna(0, inplace=True)

In [45]:
product_login.to_csv('./FE_data/product_login.csv', index=None)
del product_login, login_7d_info, login_info_7d, matrix, matrix_1, temp, product_order, product_info_7d
gc.collect()

20